In [1]:
import eikon as ek
from dataquery import *
from datetime import *
import pandas as pd
import math
import numpy as np

#Authenticate Session
ek.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

#Use Screener to get the RIC of Every Primary NYSE Stock
SCREEN.universe = Equity(active=True,public=True,primary=True,)
SCREEN.conditions = IN('TR.ExchangeMarketIdCode', "XNYS")
query = SCREEN.query

df,err = ek.get_data(query,'TR.CompanyName')

price_data = []

#Convert RIC Column to a list, and split the list into 2. The list is too big for one request
NYSE_rics = df["Instrument"].to_list()
chunks = [NYSE_rics[x:x+50] for x in range(0, len(NYSE_rics), 50)]
chunks

for i in range(0,len(chunks)):
    price_data.append(ek.get_data(chunks[i],
                                  ['TR.OPENPRICE(SDate={today},EDate={beg}).date'.format(today=datetime.today().strftime('%Y-%m-%d'),beg=(datetime.today()+timedelta(-90)).strftime('%Y-%m-%d')),
                                   'TR.OPENPRICE(SDate={today},EDate={beg})'.format(today=datetime.today().strftime('%Y-%m-%d'),beg=(datetime.today()+timedelta(-90)).strftime('%Y-%m-%d')),
                                   "TR.CLOSEPRICE(SDate={yesterday},EDate={beg}).date".format(yesterday=(datetime.today()+timedelta(-1)).strftime('%Y-%m-%d'),beg=(datetime.today()+timedelta(-91)).strftime('%Y-%m-%d')),
                                   "TR.CLOSEPRICE(SDate={yesterday},EDate={beg})".format(yesterday=(datetime.today()+timedelta(-1)).strftime('%Y-%m-%d'),beg=(datetime.today()+timedelta(-91)).strftime('%Y-%m-%d'))])[0])

NYSE = pd.concat(price_data).sort_index()

#Extract Today's Open Price and the Latest Close Price, merge both data frames, and calculate the percent change from yesterdays close to todays open
NYSE = NYSE.dropna(axis = 0).reset_index()
NYSE['Percent Change'] = ((NYSE['Open Price']-NYSE['Close Price'])/NYSE["Close Price"])*100
NYSE = NYSE.sort_values('Percent Change',ascending = False).reset_index()
NYSE.columns = ['level_0','index','Instrument','Open Date','Open Price','Close Date','Close Price','Percent Change']

In [2]:
#Extract those instruments whose % Change from Close to Open is > 15%
NYSE_Big_Gainers = NYSE[NYSE['Percent Change'] >= 15]
NYSE_Big_Gainers = NYSE_Big_Gainers[['Instrument','Open Date']]
NYSE_Big_Gainers['Open Date']=NYSE_Big_Gainers['Open Date'].str[0:10]
NYSE_Big_Gainers['Start Time'] = NYSE_Big_Gainers['Open Date'] + "T13:30:00Z"
NYSE_Big_Gainers['End Time'] = NYSE_Big_Gainers['Open Date'] + "T20:01:00Z"
NYSE_Big_Gainers['Combined'] = NYSE_Big_Gainers['Instrument'] + " " + NYSE_Big_Gainers['Open Date'].astype(str)


#This dict will store a dataframe for every request made
timeseries_data = {}

#For loop to iterate over each instance of a day where a stock opened 15% higher than its close
for (_,instrument,opendate,start,end,combined) in NYSE_Big_Gainers.itertuples():
    timeseries_data[combined] = ek.get_timeseries(instrument,
                                             fields ="CLOSE",
                                             interval = 'minute',
                                             start_date = start,
                                             end_date = end,
                                             )
    timeseries_data[combined].columns = [combined]
                            
# Change the Timeseries, remove timestamp as index and make it a separate column
for key,val in timeseries_data.items():
    timeseries_data[key]=timeseries_data[key].tz_localize('Etc/Greenwich').tz_convert("US/Eastern")
    timeseries_data[key]=timeseries_data[key].tz_localize(None)
    timeseries_data[key]['Timestamp'] = timeseries_data[key].index
    timeseries_data[key].reset_index(drop=True,inplace=True)

In [3]:
# #Not all instruments have 391 rows. Add time stamp and merge dataframes on first DF that has all datapoints

for key,val in timeseries_data.items():
    timeseries_data[key]['Time'] = timeseries_data[key]['Timestamp'].dt.time
    timeseries_data[key].set_index('Time',inplace=True)

#Merge
first_key = list(timeseries_data.keys())[0]
merged_df = timeseries_data[first_key]

for key,val in timeseries_data.items():
    if key ==  first_key:
        continue
    else:
        merged_df = merged_df.merge(right = timeseries_data[key], how = 'left', left_index = True, right_index=True)

#Rename Timestamp Columns        
mdf_columns = merged_df.columns.to_list()
timestamp_cols = []

for i in range(0,len(mdf_columns)):
    if mdf_columns[i][:9]=="Timestamp":
        col_name = 'Timestamp_{}'.format(i)
        timestamp_cols.append(col_name)
        mdf_columns[i] = col_name

merged_df.columns = mdf_columns

for key,val in merged_df.items():
    if key[:9] != "Timestamp":
        merged_df[key].fillna(method='ffill',inplace=True)
        merged_df[key].fillna(method='bfill',inplace=True)

timestamp_cols = []
for i in merged_df.columns:
    if i[:9] == 'Timestamp':
        col_name = i
        timestamp_cols.append(col_name)

ts_dict = {}
for i in timestamp_cols:
    date = merged_df[i].value_counts().index[0].date()
    ts_dict[i] = date

import datetime

for index,row in merged_df.iterrows():
    time = str(index)
    time = datetime.datetime.strptime(time,'%H:%M:%S').time()
    for key,val in ts_dict.items():
        combined = datetime.datetime.combine(val,time)
        merged_df.at[index,key] = combined

cols_fin = merged_df.columns.to_list()
for i in range(0,len(cols_fin)):
    if cols_fin[i][:9] == "Timestamp":
        cols_fin[i] = "Timestamp"
        
merged_df.columns = cols_fin
        
display(NYSE_Big_Gainers)
display(merged_df)

,Instrument,Open Date,Start Time,End Time,Combined
0,KODK.N,2020-07-28,2020-07-28T13:30:00Z,2020-07-28T20:01:00Z,KODK.N 2020-07-28
1,KODK.N,2020-07-29,2020-07-29T13:30:00Z,2020-07-29T20:01:00Z,KODK.N 2020-07-29
2,JILL.N,2020-09-01,2020-09-01T13:30:00Z,2020-09-01T20:01:00Z,JILL.N 2020-09-01
3,LUB.N,2020-09-08,2020-09-08T13:30:00Z,2020-09-08T20:01:00Z,LUB.N 2020-09-08
4,JILL.N,2020-07-16,2020-07-16T13:30:00Z,2020-07-16T20:01:00Z,JILL.N 2020-07-16
...,...,...,...,...,...
104,AMC.N,2020-09-02,2020-09-02T13:30:00Z,2020-09-02T20:01:00Z,AMC.N 2020-09-02
105,SRG.N,2020-08-07,2020-08-07T13:30:00Z,2020-08-07T20:01:00Z,SRG.N 2020-08-07
106,PEI.N,2020-08-10,2020-08-10T13:30:00Z,2020-08-10T20:01:00Z,PEI.N 2020-08-10
107,BOOT.N,2020-08-05,2020-08-05T13:30:00Z,2020-08-05T20:01:00Z,BOOT.N 2020-08-05


,KODK.N 2020-07-28,Timestamp,KODK.N 2020-07-29,Timestamp,JILL.N 2020-09-01,Timestamp,LUB.N 2020-09-08,Timestamp,JILL.N 2020-07-16,Timestamp,...,AMC.N 2020-09-02,Timestamp,SRG.N 2020-08-07,Timestamp,PEI.N 2020-08-10,Timestamp,BOOT.N 2020-08-05,Timestamp,SAIL.N 2020-08-07,Timestamp
Time,,,,,,,,,,,,,,,,,,,,,
09:31:00,9.49,2020-07-28 09:31:00,18.86,2020-07-29 09:31:00,0.9600,2020-09-01 09:31:00,1.85,2020-09-08 09:31:00,1.2200,2020-07-16 09:31:00,...,6.940,2020-09-02 09:31:00,11.84,2020-08-07 09:31:00,1.170,2020-08-10 09:31:00,23.22,2020-08-05 09:31:00,37.25,2020-08-07 09:31:00
09:32:00,10.40,2020-07-28 09:32:00,18.86,2020-07-29 09:32:00,0.9600,2020-09-01 09:32:00,1.76,2020-09-08 09:32:00,1.2900,2020-07-16 09:32:00,...,6.880,2020-09-02 09:32:00,11.84,2020-08-07 09:32:00,1.170,2020-08-10 09:32:00,23.22,2020-08-05 09:32:00,37.25,2020-08-07 09:32:00
09:33:00,9.95,2020-07-28 09:33:00,19.68,2020-07-29 09:33:00,0.9600,2020-09-01 09:33:00,1.81,2020-09-08 09:33:00,1.2900,2020-07-16 09:33:00,...,6.935,2020-09-02 09:33:00,11.84,2020-08-07 09:33:00,1.170,2020-08-10 09:33:00,23.22,2020-08-05 09:33:00,37.25,2020-08-07 09:33:00
09:34:00,9.66,2020-07-28 09:34:00,21.31,2020-07-29 09:34:00,0.9600,2020-09-01 09:34:00,2.00,2020-09-08 09:34:00,1.2900,2020-07-16 09:34:00,...,7.120,2020-09-02 09:34:00,11.84,2020-08-07 09:34:00,1.180,2020-08-10 09:34:00,23.22,2020-08-05 09:34:00,37.83,2020-08-07 09:34:00
09:35:00,9.45,2020-07-28 09:35:00,21.31,2020-07-29 09:35:00,0.9600,2020-09-01 09:35:00,1.76,2020-09-08 09:35:00,1.2900,2020-07-16 09:35:00,...,7.160,2020-09-02 09:35:00,11.84,2020-08-07 09:35:00,1.180,2020-08-10 09:35:00,23.22,2020-08-05 09:35:00,38.36,2020-08-07 09:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15:57:00,8.01,2020-07-28 15:57:00,33.21,2020-07-29 15:57:00,0.5790,2020-09-01 15:57:00,2.24,2020-09-08 15:57:00,0.8750,2020-07-16 15:57:00,...,7.025,2020-09-02 15:57:00,11.24,2020-08-07 15:57:00,1.255,2020-08-10 15:57:00,22.45,2020-08-05 15:57:00,36.30,2020-08-07 15:57:00
15:58:00,8.04,2020-07-28 15:58:00,33.21,2020-07-29 15:58:00,0.5600,2020-09-01 15:58:00,2.22,2020-09-08 15:58:00,0.8750,2020-07-16 15:58:00,...,7.035,2020-09-02 15:58:00,11.20,2020-08-07 15:58:00,1.250,2020-08-10 15:58:00,22.38,2020-08-05 15:58:00,36.27,2020-08-07 15:58:00
15:59:00,8.05,2020-07-28 15:59:00,33.21,2020-07-29 15:59:00,0.5400,2020-09-01 15:59:00,2.23,2020-09-08 15:59:00,0.8725,2020-07-16 15:59:00,...,7.030,2020-09-02 15:59:00,11.15,2020-08-07 15:59:00,1.250,2020-08-10 15:59:00,22.40,2020-08-05 15:59:00,36.20,2020-08-07 15:59:00
